In [5]:
from optionalyzer.blackscholes import BlackScholes
import pandas as pd
import datetime
import numpy as np
import yfinance

## BS

In [2]:
tesla = yfinance.Ticker("TSLA")

#Load at the money options
options = tesla.option_chain(date="2023-04-21")
calls = options.calls
puts = options.puts

spot_price = tesla.history(period="1d")['Close'][0]
print(spot_price)

#At the money options
calls = calls[((calls['strike'] - spot_price) < 10) & ((calls['strike'] - spot_price) > -10)]
puts = puts[((puts['strike'] - spot_price) < 10) & ((puts['strike'] - spot_price) > -10)]


133.4199981689453


In [3]:
calls = calls[["lastTradeDate", "strike", "lastPrice", "percentChange"]]
puts = puts[["lastTradeDate", "strike", "lastPrice", "percentChange"]]

In [4]:
calls

,lastTradeDate,strike,lastPrice,percentChange
11,2023-01-06 20:59:04+00:00,105.00,22.80,8.830542
12,2023-01-06 19:44:25+00:00,108.33,21.50,11.979162
13,2023-01-06 20:54:40+00:00,110.00,20.10,10.136989
14,2023-01-06 20:48:27+00:00,115.00,17.45,9.473032
15,2023-01-06 19:13:27+00:00,116.67,16.70,10.231030
16,2023-01-06 20:56:07+00:00,120.00,15.45,11.471863


In [5]:
puts

,lastTradeDate,strike,lastPrice,percentChange
11,2023-01-06 20:52:15+00:00,105.00,13.20,-6.382982
12,2023-01-06 20:59:25+00:00,108.33,14.75,-4.220777
13,2023-01-06 20:56:54+00:00,110.00,15.52,-5.939391
14,2023-01-06 20:54:55+00:00,115.00,18.00,-5.013197
15,2023-01-06 20:44:24+00:00,116.67,18.97,-4.912288
16,2023-01-06 20:57:55+00:00,120.00,20.74,-4.993130


In [6]:
strikes = calls['strike'].values
call_prices = calls['lastPrice'].values

put_prices = puts['lastPrice'].values


In [7]:
bs = BlackScholes()

In [8]:
df = pd.read_csv("../../Data/Stocks/nasdaq200/TSLA.csv", parse_dates=["Date"], index_col="Date")
df = df[df.index>=datetime.datetime(2016,1,1)]
return_ = df["Close"].pct_change(1).dropna()
log_return = np.log(1+return_)
sigma = log_return.std()*np.sqrt(252)
print(sigma)

0.5854294125474527


In [9]:
today = datetime.datetime.today()
april_expiry_date = datetime.datetime(today.year, 4, 21)

num_days = (april_expiry_date - today).days
print(num_days)

102


In [10]:
S = spot_price
id_ = 1
K = strikes[id_]
r = 0.0342
tau = num_days/252

call_price = bs.call(S,K,r,sigma, tau)
put_price = bs.put(S,K,r,sigma, tau)

print("Calculated Values:", call_price, put_price)
print("Trading Values:", call_prices[id_], put_prices[id_])

Calculated Values: 19.537864858888383 13.31860220954642
Trading Values: 21.5 14.75


In [11]:
S = spot_price
id_ = 4
K = strikes[id_]
r = 0.0342
tau = num_days/252

call_price = bs.call(S,K,r,sigma, tau)
put_price = bs.put(S,K,r,sigma, tau)

print("Calculated Values:", call_price, put_price)
print("Trading Values:", call_prices[id_], put_prices[id_])

Calculated Values: 15.866494878394235 17.872578201019053
Trading Values: 16.7 18.97


In [12]:
call_volas = []
put_volas = []

for i in range(len(strikes)):
    call_vola =  bs.implied_volatility(call_prices[i], S=S, K=strikes[i], r=r, tau=tau, verbose=0, option_type="call")
    put_vola =  bs.implied_volatility(put_prices[i], S=S, K=strikes[i], r=r, tau=tau, verbose=0, option_type="put")

    call_volas.append(call_vola)
    put_volas.append(put_vola)

In [13]:
call_volas

[0.646726641414932,
 0.6578991288547522,
 0.6346324270076054,
 0.6173089636017863,
 0.6147819651193667,
 0.6159793391940399]

In [14]:
put_volas

[0.6430543993629403,
 0.638280587853104,
 0.6343485117795314,
 0.6240839897173708,
 0.6240864560841277,
 0.6159687051390069]

## Option Chain

In [6]:
nifty = yfinance.Ticker("^NSEI")

In [7]:
nifty.history(period="1d")['Close'][0]

17465.80078125

In [9]:
nifty.options

()

In [9]:
nifty.option_chain(date="2023-03-02")

ValueError: Expiration `2023-04-30` cannot be found. Available expiration are: []

In [1]:
import requests
import json
import pandas as pd
import bs4 as bs
import datetime
from optionalyzer.blackscholes import OptionChain

In [2]:
iv = OptionChain("banknifty")

In [14]:
iv.option_chain(expiry_date="09-03-2023")

ValueError: Status code 404. Try again.

In [4]:
chain = iv.option_chain("2023-02-23", 20)

Loading Data
Cleaning Data


In [5]:
iv.get_option_chain()

,index,Call,Strike,Put
0,26,1959.95,41600.0,398.15
1,27,1898.25,41700.0,433.20
2,28,1837.70,41800.0,460.75
3,29,1295.20,41900.0,493.35
4,30,1223.25,42000.0,525.70
5,31,1158.40,42100.0,565.70
6,32,1103.45,42200.0,602.10
7,33,1042.95,42300.0,642.30
8,34,982.60,42400.0,685.55
9,35,930.65,42500.0,728.30


In [9]:
res = requests.get("https://groww.in/options/nifty?expiry=2023-01-25")
res.status_code

200

In [10]:
res.status_code == 200

True

In [12]:
chain = pd.read_html(res.content)[0]

In [13]:
chain

,OI (lots),CALL PRICE,STRIKE PRICE,PUT PRICE,OI (lots).1
0,1240.00%,"₹3,309.850.00 (0.00%)",14750.0,₹0.450.00 (0.00%),"1,5380.00%"
1,10.00%,"₹3,241.25+5.40 (0.17%)",14800.0,₹0.450.00 (0.00%),3650.00%
2,--,"₹3,191.25+44.05 (1.40%)",14850.0,₹0.450.00 (0.00%),--
3,70.00%,"₹3,141.30+141.30 (4.71%)",14900.0,₹0.600.00 (0.00%),750.00%
4,--,"₹3,091.35+38.40 (1.26%)",14950.0,₹0.500.00 (0.00%),--
...,...,...,...,...,...
116,--,₹0.700.00 (0.00%),20550.0,"₹2,503.55-89.50 (3.45%)",--
117,--,₹0.850.00 (0.00%),20600.0,"₹2,553.50-83.95 (3.18%)",--
118,--,₹0.850.00 (0.00%),20650.0,"₹2,603.45-78.55 (2.93%)",--
119,"2,6950.00%",₹0.800.00 (0.00%),20700.0,"₹2,653.40+26.10 (0.99%)",390.00%


In [14]:
chain = chain[(chain["OI (lots)"]!="--") & (chain["OI (lots).1"]!="--")]
chain = chain[["CALL PRICE", "STRIKE PRICE", "PUT PRICE"]]
chain.head()

,CALL PRICE,STRIKE PRICE,PUT PRICE
0,"₹3,309.850.00 (0.00%)",14750.0,₹0.450.00 (0.00%)
1,"₹3,241.25+5.40 (0.17%)",14800.0,₹0.450.00 (0.00%)
3,"₹3,141.30+141.30 (4.71%)",14900.0,₹0.600.00 (0.00%)
5,"₹3,051.600.00 (0.00%)",15000.0,₹0.400.00 (0.00%)
7,"₹2,941.50-179.50 (5.75%)",15100.0,₹0.550.00 (0.00%)


In [17]:
chain[chain["STRIKE PRICE"] == 17550.0]

,CALL PRICE,STRIKE PRICE,PUT PRICE
56,₹510.000.00 (0.00%),17550.0,₹3.850.00 (0.00%)


In [18]:
def clean_price(x):
    if "+" in x:
        x = x.split("+")[0]
    elif "-" in x:
        x = x.split("-")[0]
    else:
        x = x.split("0.00 (")[0].strip()
    x = x.replace("₹","")
    x = x.replace(",","")
    return float(x)

In [19]:
chain["CALL PRICE"] = chain["CALL PRICE"].apply(clean_price)
chain["PUT PRICE"] = chain["PUT PRICE"].apply(clean_price)
chain.head()

,CALL PRICE,STRIKE PRICE,PUT PRICE
0,3309.85,14750.0,0.45
1,3241.25,14800.0,0.45
3,3141.30,14900.0,0.60
5,3051.60,15000.0,0.40
7,2941.50,15100.0,0.55


In [20]:
chain[chain["STRIKE PRICE"] == 17550.0]

,CALL PRICE,STRIKE PRICE,PUT PRICE
56,510.0,17550.0,3.85


In [116]:
def option_chain(option="nifty", expiry_date=""):
    if expiry_date=="":
        expiry_date = datetime.datetime.today().strftime("%Y-%m-%d")
    url = f"https://groww.in/options/{option}?expiry={expiry_date}"
    print(url)
    chain = pd.read_html(f"https://groww.in/options/{option}?expiry={expiry_date}")[0]
    print("Data loaded")
    chain = chain[(chain["OI (lots)"]!="--") & (chain["OI (lots).1"]!="--")]
    chain = chain[["CALL PRICE", "STRIKE PRICE", "PUT PRICE"]]
    chain["CALL PRICE"] = chain["CALL PRICE"].apply(clean_price)
    chain["PUT PRICE"] = chain["PUT PRICE"].apply(clean_price)
    chain.columns = ["Call", "Strike", "Put"]
    return chain

In [117]:
chain_f = option_chain(expiry_date="2023-01-25")
chain_f.head()

https://groww.in/options/nifty?expiry=2023-01-25
Data loaded


,Call,Strike,Put
0,3309.85,14750.0,0.45
1,3241.25,14800.0,0.45
3,3141.30,14900.0,0.60
5,3051.60,15000.0,0.40
7,2941.50,15100.0,0.55


In [146]:
yfinance.download("^NSEI", period="1d", interval="1d")

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-20,18115.599609,18145.449219,18016.199219,18027.650391,18027.650391,237200


In [157]:
nifty = yfinance.Ticker("^nsei")

In [159]:
spot_price = nifty.history("1d", "1d")["Close"].values[0]

In [163]:
strikes = chain_f["Strike"]
strikes

0      14750.0
1      14800.0
3      14900.0
5      15000.0
7      15100.0
        ...   
111    20300.0
113    20400.0
115    20500.0
119    20700.0
120    20750.0
Name: Strike, Length: 100, dtype: float64

In [166]:
u_chain = chain_f[strikes<=spot_price]
l_chain = chain_f[strikes>spot_price]

In [176]:
u_chain[-5:], l_chain[:5]

(      Call   Strike    Put
 61  272.55  17800.0  18.50
 62  230.05  17850.0  26.25
 63  191.50  17900.0  37.10
 64  155.35  17950.0  50.85
 65  122.70  18000.0  68.50,
      Call   Strike     Put
 66  94.25  18050.0   89.90
 67  70.05  18100.0  116.10
 68  51.25  18150.0  146.70
 69  35.90  18200.0  181.75
 70  24.60  18250.0  220.40)

In [173]:
pd.concat([u_chain[-5:], l_chain[:5]])

,Call,Strike,Put
61,272.55,17800.0,18.50
62,230.05,17850.0,26.25
63,191.50,17900.0,37.10
64,155.35,17950.0,50.85
65,122.70,18000.0,68.50
66,94.25,18050.0,89.90
67,70.05,18100.0,116.10
68,51.25,18150.0,146.70
69,35.90,18200.0,181.75
70,24.60,18250.0,220.40


## IV

In [1]:
import datetime
import pandas as pd
from optionalyzer.blackscholes import OptionChain, BlackScholes
import warnings
warnings.filterwarnings("ignore")

In [2]:
oc = OptionChain("niftybank")
chain = oc.option_chain("2023-01-25", 20)

INFO:optionalyzer.bs_logger:Calling `self._chain`.
INFO:optionalyzer.bs_logger:Calling `self.__chain`.


In [3]:
chain

,Call,Strike,Put
0,943.30,42000.0,29.85
1,852.30,42100.0,36.15
2,760.30,42200.0,45.55
3,670.50,42300.0,57.05
4,586.80,42400.0,71.25
5,505.35,42500.0,89.15
6,426.75,42600.0,111.60
7,358.95,42700.0,140.05
8,290.05,42800.0,175.00
9,232.20,42900.0,216.85


In [4]:
call_iv = oc.calculate_iv(expiry_date="2023-01-25", option_type="call")
put_iv = oc.calculate_iv(expiry_date="2023-01-25", option_type="put")

INFO:optionalyzer.bs_logger:Defining variables.


Calculating Implied Volatility


INFO:optionalyzer.bs_logger:Defining variables.


Calculating Implied Volatility


In [5]:
call_iv, put_iv

(0.23920623607192992, 0.28413852964637126)

In [5]:
oc.get_option_chain()

,Call,Strike,Put,Call IV,Put IV
0,510.00,17550.0,3.85,0.463736,0.287632
1,457.95,17600.0,5.15,0.419590,0.277756
2,410.50,17650.0,6.60,0.397095,0.265008
3,363.00,17700.0,9.40,0.372203,0.258159
4,316.75,17750.0,13.05,0.349946,0.249807
5,272.55,17800.0,18.50,0.331828,0.243273
6,230.05,17850.0,26.25,0.314759,0.237711
7,191.50,17900.0,37.10,0.304015,0.233362
8,155.35,17950.0,50.85,0.292724,0.227422
9,122.70,18000.0,68.50,0.282645,0.221270


In [6]:
tau = (datetime.datetime(2023,1,25) - datetime.datetime.today()).days/356
tau

0.0028089887640449437

In [10]:
bs = BlackScholes()

In [29]:
S = oc._OptionChain__spot_price
r = 0.0744
ivs_call = []
for i in range(len(chain)):
    option_price = chain.iloc[i]["Call"]
    K = chain.iloc[i]["Strike"]
    iv = bs.implied_volatility(option_price, S=S, K=K, tau=tau, option_type="call", r=r, verbose= 0)
    ivs_call.append(iv)

In [30]:
S = oc._OptionChain__spot_price
ivs_put = []
for i in range(len(chain)):
    option_price = chain.iloc[i]["Put"]
    K = chain.iloc[i]["Strike"]
    iv = bs.implied_volatility(option_price, S=S, K=K, tau=tau, option_type="put", r=r, verbose= 0)
    ivs_put.append(iv)

In [31]:
ivs_call

[0.4575960130889868,
 0.4140026836986674,
 0.3918224655615509,
 0.36727153661428585,
 0.34532365132649956,
 0.3274656432860636,
 0.31064155999438875,
 0.3000615142089859,
 0.2889363777040177,
 0.27900490154861357,
 0.270796371890648,
 0.2635240137516945,
 0.2601650273768537,
 0.25580714366417334,
 0.2534436984330829,
 0.2514486870454162,
 0.25048703915408776,
 0.2541312558826606,
 0.2615671801049782,
 0.27624947120752025]

In [32]:
ivs_put

[0.2841080384046109,
 0.27435698396354535,
 0.2617695548710392,
 0.25500970929817746,
 0.24676558174871627,
 0.2403184078360003,
 0.23483299994319604,
 0.23054730979773755,
 0.22469305303822498,
 0.2186339524083159,
 0.21047636351458118,
 0.20164700723284637,
 0.18963924400250082,
 0.1719335103847358,
 0.1279919469972001,
 0.006018821443261914,
 0.05711259828020953,
 0.06840909084350971,
 0.08258759144633089,
 0.08805308760949895]

In [34]:
import numpy as np

np.median(ivs_call), np.median(ivs_put)

(0.27762718637806694, 0.21455515796144853)

## NSE

In [18]:
import requests
import pandas as pd

In [38]:
url = "https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY"
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, '
                         'like Gecko) '
                         'Chrome/80.0.3987.149 Safari/537.36',
           'accept-language': 'en,gu;q=0.9,hi;q=0.8', 'accept-encoding': 'gzip, deflate, br'}
r = requests.get(url, headers=headers)

In [39]:
json_parsed = r.json()
json_parsed.keys()

dict_keys(['records', 'filtered'])

In [40]:
data = pd.DataFrame(json_parsed['records']['data'])
data

,strikePrice,expiryDate,CE,PE
0,8500,29-Jun-2023,"{'strikePrice': 8500, 'expiryDate': '29-Jun-20...",NaN
1,9500,29-Jun-2023,"{'strikePrice': 9500, 'expiryDate': '29-Jun-20...",NaN
2,10000,29-Jun-2023,NaN,"{'strikePrice': 10000, 'expiryDate': '29-Jun-2..."
3,10500,29-Jun-2023,NaN,"{'strikePrice': 10500, 'expiryDate': '29-Jun-2..."
4,11000,28-Dec-2023,"{'strikePrice': 11000, 'expiryDate': '28-Dec-2...","{'strikePrice': 11000, 'expiryDate': '28-Dec-2..."
...,...,...,...,...
715,23000,31-Dec-2026,"{'strikePrice': 23000, 'expiryDate': '31-Dec-2...","{'strikePrice': 23000, 'expiryDate': '31-Dec-2..."
716,23000,24-Jun-2027,"{'strikePrice': 23000, 'expiryDate': '24-Jun-2...","{'strikePrice': 23000, 'expiryDate': '24-Jun-2..."
717,23000,30-Dec-2027,"{'strikePrice': 23000, 'expiryDate': '30-Dec-2...","{'strikePrice': 23000, 'expiryDate': '30-Dec-2..."
718,24000,29-Mar-2023,"{'strikePrice': 24000, 'expiryDate': '29-Mar-2...","{'strikePrice': 24000, 'expiryDate': '29-Mar-2..."


In [41]:
expiry_dates = data['expiryDate'].unique()
expiry_dates

array(['29-Jun-2023', '28-Dec-2023', '31-Dec-2026', '29-Mar-2023',
       '25-Jun-2026', '26-Dec-2024', '26-Jun-2025', '24-Dec-2025',
       '24-Jun-2027', '30-Dec-2027', '27-Jun-2024', '28-Sep-2023',
       '25-Jan-2023', '23-Feb-2023', '09-Feb-2023', '02-Feb-2023',
       '16-Feb-2023', '02-Mar-2023'], dtype=object)

In [59]:
data_one = data[data["expiryDate"] == '02-Feb-2023']

In [60]:
data_f = data_one.dropna(axis=0)

In [61]:
data_f

,strikePrice,expiryDate,CE,PE
106,16300,02-Feb-2023,"{'strikePrice': 16300, 'expiryDate': '02-Feb-2...","{'strikePrice': 16300, 'expiryDate': '02-Feb-2..."
108,16350,02-Feb-2023,"{'strikePrice': 16350, 'expiryDate': '02-Feb-2...","{'strikePrice': 16350, 'expiryDate': '02-Feb-2..."
115,16400,02-Feb-2023,"{'strikePrice': 16400, 'expiryDate': '02-Feb-2...","{'strikePrice': 16400, 'expiryDate': '02-Feb-2..."
122,16450,02-Feb-2023,"{'strikePrice': 16450, 'expiryDate': '02-Feb-2...","{'strikePrice': 16450, 'expiryDate': '02-Feb-2..."
131,16500,02-Feb-2023,"{'strikePrice': 16500, 'expiryDate': '02-Feb-2...","{'strikePrice': 16500, 'expiryDate': '02-Feb-2..."
...,...,...,...,...
656,20100,02-Feb-2023,"{'strikePrice': 20100, 'expiryDate': '02-Feb-2...","{'strikePrice': 20100, 'expiryDate': '02-Feb-2..."
660,20150,02-Feb-2023,"{'strikePrice': 20150, 'expiryDate': '02-Feb-2...","{'strikePrice': 20150, 'expiryDate': '02-Feb-2..."
664,20200,02-Feb-2023,"{'strikePrice': 20200, 'expiryDate': '02-Feb-2...","{'strikePrice': 20200, 'expiryDate': '02-Feb-2..."
667,20250,02-Feb-2023,"{'strikePrice': 20250, 'expiryDate': '02-Feb-2...","{'strikePrice': 20250, 'expiryDate': '02-Feb-2..."


In [62]:
data_f['PE'].apply(pd.Series).T

,106,108,115,122,131,138,144,150,159,160,...,624,629,633,637,652,656,660,664,667,670
strikePrice,16300,16350,16400,16450,16500,16550,16600,16650,16700,16750,...,19850,19900,19950,20000,20050,20100,20150,20200,20250,20300
expiryDate,02-Feb-2023,02-Feb-2023,02-Feb-2023,02-Feb-2023,02-Feb-2023,02-Feb-2023,02-Feb-2023,02-Feb-2023,02-Feb-2023,02-Feb-2023,...,02-Feb-2023,02-Feb-2023,02-Feb-2023,02-Feb-2023,02-Feb-2023,02-Feb-2023,02-Feb-2023,02-Feb-2023,02-Feb-2023,02-Feb-2023
underlying,NIFTY,NIFTY,NIFTY,NIFTY,NIFTY,NIFTY,NIFTY,NIFTY,NIFTY,NIFTY,...,NIFTY,NIFTY,NIFTY,NIFTY,NIFTY,NIFTY,NIFTY,NIFTY,NIFTY,NIFTY
identifier,OPTIDXNIFTY02-02-2023PE16300.00,OPTIDXNIFTY02-02-2023PE16350.00,OPTIDXNIFTY02-02-2023PE16400.00,OPTIDXNIFTY02-02-2023PE16450.00,OPTIDXNIFTY02-02-2023PE16500.00,OPTIDXNIFTY02-02-2023PE16550.00,OPTIDXNIFTY02-02-2023PE16600.00,OPTIDXNIFTY02-02-2023PE16650.00,OPTIDXNIFTY02-02-2023PE16700.00,OPTIDXNIFTY02-02-2023PE16750.00,...,OPTIDXNIFTY02-02-2023PE19850.00,OPTIDXNIFTY02-02-2023PE19900.00,OPTIDXNIFTY02-02-2023PE19950.00,OPTIDXNIFTY02-02-2023PE20000.00,OPTIDXNIFTY02-02-2023PE20050.00,OPTIDXNIFTY02-02-2023PE20100.00,OPTIDXNIFTY02-02-2023PE20150.00,OPTIDXNIFTY02-02-2023PE20200.00,OPTIDXNIFTY02-02-2023PE20250.00,OPTIDXNIFTY02-02-2023PE20300.00
openInterest,703,79,174,48,2311,104,359,48,1988,270,...,0,0,0,0,0,0,0,0,0,0
changeinOpenInterest,144,-16,86,-17,1068,-31,50,-48,1452,203,...,0,0,0,0,0,0,0,0,0,0
pchangeinOpenInterest,25.760286,-16.842105,97.727273,-26.153846,85.921158,-22.962963,16.18123,-50.0,270.895522,302.985075,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
totalTradedVolume,869,483,500,44,5815,305,1466,321,4803,614,...,0,0,0,0,0,0,0,0,0,0
impliedVolatility,24.51,24.92,23.74,23.35,22.85,22.28,21.87,21.31,20.96,20.32,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
lastPrice,3.3,4.5,3.85,4.15,4.35,4.45,4.8,4.95,5.45,5.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
